In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import warnings
import time
warnings.filterwarnings(action='ignore')

In [5]:
# 드라이버 불러오기
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [10]:
# 메인 페이지
url = 'https://biz.koke.kr/products?filters=%7B%22categoryGroup%22%3A%22WholeBean%22%7D'
driver.get(url) # go to website
time.sleep(1)

In [11]:
links = [] # 제품 링크 리스트

# 약 4분 30초 소요
for i in range(32):
    raw = driver.find_elements(By.CSS_SELECTOR,"div.flex.flex-col.w-full.lg\:w-3\/4")
    link_elements = raw[0].find_elements(By.TAG_NAME,'a') 
    href_links = [link.get_attribute('href') for link in link_elements]
    links = links + href_links # 현재 페이지에 있는 상품 가져오기
    time.sleep(3)

    if i is not 31:
        button = driver.find_elements(By.CSS_SELECTOR ,"button.text-gray-500.bg-white.border.border-gray-200.koke-button")
        button[-1].click()
        time.sleep(5)
    
    print('현제 페이지(i+1): ',i+1,' 링크 수: ',len(href_links),' 총 링크 수: ',len(links))

link = set(links)
print(len(links)) # 총 556개


현제 페이지(i+1):  1  링크 수:  18  총 링크 수:  18
현제 페이지(i+1):  2  링크 수:  18  총 링크 수:  36
현제 페이지(i+1):  3  링크 수:  18  총 링크 수:  54
현제 페이지(i+1):  4  링크 수:  18  총 링크 수:  72
현제 페이지(i+1):  5  링크 수:  18  총 링크 수:  90
현제 페이지(i+1):  6  링크 수:  18  총 링크 수:  108
현제 페이지(i+1):  7  링크 수:  18  총 링크 수:  126
현제 페이지(i+1):  8  링크 수:  18  총 링크 수:  144
현제 페이지(i+1):  9  링크 수:  18  총 링크 수:  162
현제 페이지(i+1):  10  링크 수:  18  총 링크 수:  180
현제 페이지(i+1):  11  링크 수:  18  총 링크 수:  198
현제 페이지(i+1):  12  링크 수:  18  총 링크 수:  216
현제 페이지(i+1):  13  링크 수:  18  총 링크 수:  234
현제 페이지(i+1):  14  링크 수:  18  총 링크 수:  252
현제 페이지(i+1):  15  링크 수:  18  총 링크 수:  270
현제 페이지(i+1):  16  링크 수:  18  총 링크 수:  288
현제 페이지(i+1):  17  링크 수:  18  총 링크 수:  306
현제 페이지(i+1):  18  링크 수:  18  총 링크 수:  324
현제 페이지(i+1):  19  링크 수:  18  총 링크 수:  342
현제 페이지(i+1):  20  링크 수:  18  총 링크 수:  360
현제 페이지(i+1):  21  링크 수:  18  총 링크 수:  378
현제 페이지(i+1):  22  링크 수:  18  총 링크 수:  396
현제 페이지(i+1):  23  링크 수:  18  총 링크 수:  414
현제 페이지(i+1):  24  링크 수:  18  총 링크 수:  432
현제 페이지

In [13]:

len(link)

559

In [14]:
import pickle

# Pickle dump the set of links to a file
with open("links_559.pkl", "wb") as file:
    pickle.dump(link, file)

# Load the set of links from the file
with open("links_559.pkl", "rb") as file:
    loaded_links = pickle.load(file)

# Print the loaded set of links
print("Loaded links:", len(loaded_links))

Loaded links: 559


In [63]:
def get_roastery(url):
    # 제품 사이트로 이동
    driver.get(url)
    time.sleep(3)
    # 정보 블록
    raw = driver.find_element(By.CSS_SELECTOR,"div.md\:pt-\[4rem\]")
    
    # 원두 이름
    bean_name = driver.find_element(By.CSS_SELECTOR,"div.flex.flex-col").find_element(By.TAG_NAME,'h1').text
    # 로스터리 이름
    roastery = raw.find_element(By.TAG_NAME,'h2').text
    # 소개글
    temp = raw.find_element(By.CSS_SELECTOR,"div.w-full.md\:w-168.md\:koke-word-keep-all")
    introduction = temp.find_element(By.TAG_NAME,'p').text
    # 주소
    address = temp.find_elements(By.TAG_NAME,'li')[-1].find_element(By.TAG_NAME,'p').text
    # 이미지
    img = raw.find_elements(By.TAG_NAME,'img')[-1].get_attribute('src')
    
    return [bean_name,roastery,introduction,address,img]


In [ ]:
temp_list = []
for link in links:
    temp = get_roastery(link)
    temp_list.append(temp)

In [76]:
df = pd.DataFrame(temp_list,
                  columns=['name','roastery','info','address','img'])
df.tail()

,name,roastery,info,address,img
551,파푸아뉴기니 AA 쿠아 블루마운틴 풀리워시드,커스프 커피,커스프는 스텀프타운에서 시작한 20년 경력의 대표 로스터 JAY가 창업한 뉴욕 브루...,서울시 강남구 학동로31길 12 7F,https://d2wosiipoa41qn.cloudfront.net/UxPzmLPG...
552,케냐 뚱구리 피베리,180커피로스터스,180커피로스터스는 180도 생각의 전환이라는 뜻을 가지고 있습니다.\n우리는 단순...,경기 성남시 분당구 문정로144번길 4,https://d2wosiipoa41qn.cloudfront.net/j5QPmenj...
553,디카페인 스프링 블렌드,커스프 커피,커스프는 스텀프타운에서 시작한 20년 경력의 대표 로스터 JAY가 창업한 뉴욕 브루...,서울시 강남구 학동로31길 12 7F,https://d2wosiipoa41qn.cloudfront.net/UxPzmLPG...
554,브라질 핀할 카티구아,180커피로스터스,180커피로스터스는 180도 생각의 전환이라는 뜻을 가지고 있습니다.\n우리는 단순...,경기 성남시 분당구 문정로144번길 4,https://d2wosiipoa41qn.cloudfront.net/j5QPmenj...
555,콜롬비아 수프리모 후일라 산 비센테 라 플라타,커스프 커피,커스프는 스텀프타운에서 시작한 20년 경력의 대표 로스터 JAY가 창업한 뉴욕 브루...,서울시 강남구 학동로31길 12 7F,https://d2wosiipoa41qn.cloudfront.net/UxPzmLPG...


In [78]:
# df.to_csv('roastery.csv')

# 로스터리 이미지 다운로드


In [79]:
df = pd.read_csv('roastery.csv').iloc[:,1:]
df.head()

,name,roastery,info,address,img
0,마르께,선빌리지 커피,'Fresh Roasted Coffee Company' 선빌리지커피는 신선한 뉴크롭...,경북 경산시 와촌면 갓바위로 67-2,https://d2wosiipoa41qn.cloudfront.net/O-hzQT0A...
1,아헨 Aachen,크레센도 커피 로스터스,Life is perfect with a cup of coffee and some ...,"경기도 성남시 중원구 둔촌대로 400, 5층 511호",https://d2wosiipoa41qn.cloudfront.net/fPsdgknw...
2,진양조,스탠딩커피,"근사한 커피한잔을 위해서는 훌륭한 생두의 선별, 생두의 장점을 극대화한 로스팅, 원...",경기도 포천시 소흘읍 죽엽산로298번길 56,https://d2wosiipoa41qn.cloudfront.net/9a6odqLa...
3,클래식 에스프레소,리플렉트 커피 로스터스,'우리를 커피에 비추다. 우리의 색과 진심을 커피에 반영하다'\n품질 좋은 재료를 ...,서울시 서초구 효령로 72길 57,https://d2wosiipoa41qn.cloudfront.net/UDoFsmm2...
4,올드타운로드 하우스블랜드,컨트리사이드,2020년 어느 외곽지역에서 컨트리사이드는 커피 관련 다양한 경험과 장점을 바탕으로...,경기도 고양시 덕양구 행당로 1 4층 401호,https://d2wosiipoa41qn.cloudfront.net/juSlADg5...


In [80]:
filtered_df = df.drop_duplicates(subset='roastery', keep='first').loc[:,['roastery','img']].reset_index().iloc[:,1:]
filtered_df.head()

,roastery,img
0,선빌리지 커피,https://d2wosiipoa41qn.cloudfront.net/O-hzQT0A...
1,크레센도 커피 로스터스,https://d2wosiipoa41qn.cloudfront.net/fPsdgknw...
2,스탠딩커피,https://d2wosiipoa41qn.cloudfront.net/9a6odqLa...
3,리플렉트 커피 로스터스,https://d2wosiipoa41qn.cloudfront.net/UDoFsmm2...
4,컨트리사이드,https://d2wosiipoa41qn.cloudfront.net/juSlADg5...


In [ ]:
import requests

for i in range(len(filtered_df)):
    roastery = filtered_df.loc[i,'roastery']
    img_url = filtered_df.loc[i,'img']
    
    response = requests.get(img_url)
    
    if response.status_code == 200:
        with open('roastery_img/'+str(roastery)+'.jpg', 'wb') as file:
            file.write(response.content)
            print(i,'번째 ',roastery, " Image downloaded successfully.")
    else:
        print(roastery, f" Failed to download the image. Status code: {response.status_code}")


## 정보 추가 (원두 추가)

In [3]:
df = pd.read_csv('roastery.csv').iloc[:,1:]
df.head()

,name,roastery,info,address,img
0,마르께,선빌리지 커피,'Fresh Roasted Coffee Company' 선빌리지커피는 신선한 뉴크롭...,경북 경산시 와촌면 갓바위로 67-2,https://d2wosiipoa41qn.cloudfront.net/O-hzQT0A...
1,아헨 Aachen,크레센도 커피 로스터스,Life is perfect with a cup of coffee and some ...,"경기도 성남시 중원구 둔촌대로 400, 5층 511호",https://d2wosiipoa41qn.cloudfront.net/fPsdgknw...
2,진양조,스탠딩커피,"근사한 커피한잔을 위해서는 훌륭한 생두의 선별, 생두의 장점을 극대화한 로스팅, 원...",경기도 포천시 소흘읍 죽엽산로298번길 56,https://d2wosiipoa41qn.cloudfront.net/9a6odqLa...
3,클래식 에스프레소,리플렉트 커피 로스터스,'우리를 커피에 비추다. 우리의 색과 진심을 커피에 반영하다'\r\n품질 좋은 재료...,서울시 서초구 효령로 72길 57,https://d2wosiipoa41qn.cloudfront.net/UDoFsmm2...
4,올드타운로드 하우스블랜드,컨트리사이드,2020년 어느 외곽지역에서 컨트리사이드는 커피 관련 다양한 경험과 장점을 바탕으로...,경기도 고양시 덕양구 행당로 1 4층 401호,https://d2wosiipoa41qn.cloudfront.net/juSlADg5...


In [7]:
'와이칸 다크' in list(df.name)

True